In [1]:
import numpy as np

In [2]:
def EstimateCorrespondences(X,Y,t,R,d_max):
    
    C = np.empty((0,2), dtype=int)
    # Transform the points
    transposed_x = np.dot(X,R) + t
    # Find the indexes of the least-squared sense. 
    for i in range(transposed_x.shape[0]):
        norm = np.linalg.norm(Y-transposed_x[i], axis=1)
        y = np.argmin(norm)
        if norm[y] < d_max :
            C = np.vstack((C, [i,y]))
    return C

In [3]:
def ComputeOptimalRigidRegistration(X,Y,C) :

    # Calculate the point cloud centroid of X and Y
    x_centroid = np.mean(X[C[:,0]], axis=0)
    y_centroid = np.mean(Y[C[:,1]], axis=0)

    # Calculate the deviation of X and Y from the centroid
    x_deviation = X[C[:,0]] - x_centroid
    y_deviation = Y[C[:,1]] - y_centroid

    # Calculate the covariance matrix
    W = np.dot(x_deviation.T, y_deviation)

    # Calculate the SVD
    u, s , vh = np.linalg.svd(W)


    # Construct the optimal rotation :
    Rot = np.dot(u,vh)

    # Optimal translation :
    trans =  y_centroid - np.dot(x_centroid,Rot) 

    return Rot, trans

In [4]:
def ICP_alg(X , Y , to, Ro, d_max, max_iter):

    iter = 0
    while iter != max_iter : 
        C = EstimateCorrespondences(X,Y,to,Ro,d_max)
        R , t = ComputeOptimalRigidRegistration(X,Y,C)
        to = t
        Ro = R
        iter += 1
    return t , R , C


In [5]:
if __name__ == "__main__":
    pcl_X = np.loadtxt("pclX.txt", dtype = float)
    pcl_Y = np.loadtxt("pclY.txt", dtype = float)
    

    t = np.zeros((1,3))

    R = np.array([[1,0,0],[0,1,0],[0,0,1]])
    d_max = 0.25
    iter = 30
    
    t , R, C = ICP_alg(pcl_X,pcl_Y,t,R,d_max,iter)


In [6]:
print(R)
print(t)
trans = np.dot(pcl_X,R) + t


s = 0
s = np.mean(np.linalg.norm(pcl_Y - trans, axis=1)**2)
RMSE = np.sqrt(s)
print(RMSE)

[[ 0.95126601  0.22323628  0.21274056]
 [-0.15043058  0.9381636  -0.31180074]
 [-0.26919069  0.26460276  0.92602471]]
[ 0.49661487 -0.29392971  0.29645004]
0.46043360699622676


In [7]:
# Scatter plot of the points
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pcl_X[:,0], pcl_X[:,1], pcl_X[:,2], c='b', marker='o', s = 0.8)
ax.scatter(pcl_Y[:,0], pcl_Y[:,1], pcl_Y[:,2], c='g', marker='o', s = 0.8)
ax.scatter(trans[:,0], trans[:,1], trans[:,2], c='r', marker='o', s = 0.8)
#Add the legend
ax.legend(['X', 'Y', 'Transformed X'])
ax.scatter
plt.show()
